Imports

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split

Read CSV

In [39]:
# Set seed
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Load the dataset
dataset = pd.read_csv("../../datasets/final_dataset.csv")

# First split: train and test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset["Text"], dataset["Label"], test_size=0.2, random_state=42, stratify=dataset["Label"]
)

# Second split: train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

Process Data

In [40]:
# Tokenize the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=100)
X_val = pad_sequences(X_val_seq, maxlen=100)
X_test = pad_sequences(X_test_seq, maxlen=100)

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)


Neural Network

In [41]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),

    Conv1D(64, 5, activation="relu", kernel_regularizer=l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.6),

    Conv1D(32, 5, activation="relu", kernel_regularizer=l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.6),

    Flatten(),
    BatchNormalization(),

    Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
    Dropout(0.7),

    Dense(32, activation="relu", kernel_regularizer=l2(0.01)),
    Dropout(0.7),

    Dense(1, activation="sigmoid")
])

# Compile with AdamW optimizer
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0002, weight_decay=0.01)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=2,
    min_lr=0.00005
)

# Train with adjusted dropout, regularization, and patience for early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50


c:\Users\tumab\miniconda3\envs\DeepLearning\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4998 - loss: 3.7101 - val_accuracy: 0.5177 - val_loss: 3.2744 - learning_rate: 2.0000e-04
Epoch 2/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5052 - loss: 3.6183 - val_accuracy: 0.5177 - val_loss: 3.1941 - learning_rate: 2.0000e-04
Epoch 3/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5188 - loss: 3.4399 - val_accuracy: 0.5023 - val_loss: 3.1141 - learning_rate: 2.0000e-04
Epoch 4/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4970 - loss: 3.2548 - val_accuracy: 0.5208 - val_loss: 3.0332 - learning_rate: 2.0000e-04
Epoch 5/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5279 - loss: 3.0948 - val_accuracy: 0.5039 - val_loss: 2.9522 - learning_rate: 2.0000e-04
Epoch 6/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5175 - loss: 3.0019 - val_accuracy: 0.4915 - val_loss: 2.8698 - learning_rate: 2.0000e-04
Epoch 7/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5147 - loss: 2.8797

Results

In [42]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}\n")

val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {val_acc:.4f}")

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9792 - loss: 0.2022
Test accuracy: 0.9766

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9679 - loss: 0.2059 
Validation accuracy: 0.9661


Benchmarking

In [43]:
# Load new data
new_data = pd.read_csv("../../datasets/validation_dataset.csv", delimiter=";")

# Tokenize and pad the new data (using the same tokenizer you trained on)
X_new_seq = tokenizer.texts_to_sequences(new_data["Text"])
X_new = pad_sequences(X_new_seq, maxlen=100)  # Ensure maxlen is consistent with your training data

# Make predictions
predictions = model.predict(X_new).flatten()

# Convert predictions to labels based on threshold
labels = ["AI" if pred > 0.5 else "Human" for pred in predictions]

# Create output DataFrame with predictions
output_df = pd.DataFrame({
    "Label": labels,
    "Prediction": predictions
})

# Load the ground truth labels (from the same dataset)
# Since the labels are in the 'Label' column, we'll compare them with predictions.
ground_truth = new_data["Label"]

# Calculate accuracy
accuracy = (output_df["Label"] == ground_truth).mean()

# Print the accuracy
print(f"Accuracy: {accuracy:.4f}")

# Merge predictions with ground truth for comparison
comparison_df = output_df.copy()
comparison_df["Label_actual"] = ground_truth

# Show misclassified samples
misclassified = comparison_df[comparison_df["Label"] != comparison_df["Label_actual"]]
print("\nMisclassified Samples:")
print(misclassified)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Accuracy: 0.6125

Misclassified Samples:
    Label  Prediction Label_actual
5   Human    0.000360           AI
9   Human    0.001497           AI
10  Human    0.031456           AI
12  Human    0.000089           AI
13  Human    0.057404           AI
22  Human    0.006830           AI
24  Human    0.083040           AI
28  Human    0.000364           AI
32     AI    0.863464        Human
35  Human    0.083501           AI
37  Human    0.005602           AI
38  Human    0.230091           AI
39  Human    0.424738           AI
41     AI    0.986609        Human
42  Human    0.013589           AI
45  Human    0.063489           AI
48     AI    0.881551        Human
51  Human    0.026293           AI
52     AI    0.507281        Human
53  Human    0.000753           AI
55  Human    0.219655           AI
57  Human    0.002944           AI
59  Human    0.001711           AI
61  Human    0.009234           AI
67  Human    0.000137           AI
69  Human 